In [2]:
# --- 0. CONNECT TO THE QUANTUM LOOM ---
import warnings
import os
import logging
import sys

# 1. Kill Standard Warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

# 2. Kill ResourceWarnings (Socket warnings from Qiskit Runtime)
warnings.filterwarnings("ignore", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 3. Kill System Logs (TensorFlow/Qiskit/Urllib3)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['QISKIT_SETTINGS'] = '{"suppress_warnings": true}'

logging.getLogger('qiskit').setLevel(logging.CRITICAL)
logging.getLogger('stevedore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)
logging.getLogger('qiskit_ibm_runtime').setLevel(logging.CRITICAL)

from qiskit_ibm_runtime import QiskitRuntimeService
import os

# Load saved account
try:
    service = QiskitRuntimeService()
    print(f"Connected to IBM Quantum: {service.channel}")
except:
    print("No saved account found.")
    token = input("Please enter your IBM Quantum API Token: ")
    service = QiskitRuntimeService(channel="ibm_quantum", token=token)

# Hardware Selection Strategy
# We prioritize the 156-qubit 'ibm_marrakesh' or 'ibm_fez' for topological fidelity.
desired_backends = ["ibm_marrakesh", "ibm_fez", "ibm_torino"]
backend = None

for name in desired_backends:
    try:
        backend = service.backend(name)
        print(f"Targeting High-Fidelity Hardware: {backend.name}")
        break
    except:
        continue

if backend is None:
    # Fallback to whatever is available
    backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
    print(f"Fallback Target: {backend.name}")

Connected to IBM Quantum: ibm_quantum_platform
Targeting High-Fidelity Hardware: ibm_marrakesh


In [3]:
# --- QUANTUM LOOM: BLACK HOLE JAMMING (SINGULARITY REFUTATION) ---
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import SamplerV2
import numpy as np

# Theory: As Mass Density (Phase Winding) increases, the vacuum should
# transition from a Superfluid (Entangled) to a Solid (Product State).
# This "Jamming" prevents infinite density (Singularity).

densities = np.linspace(0, np.pi*2, 10) # Increasing Phase Density

circuits = []
for rho in densities:
    # 5 Qubit "Star" Collapse Model
    qr = QuantumRegister(5, 'q')
    cr = ClassicalRegister(4, 'mobility') # Measure mobility of outer nodes
    qc = QuantumCircuit(qr, cr)
    
    # A. Initialize Superfluid
    qc.h(range(5))
    
    # B. Apply Gravitational Tension (The Squeeze)
    # We entangle the center (Q0) with surroundings (Q1-Q4)
    # The 'rho' parameter controls the strength of the squeeze.
    for i in range(1, 5):
        qc.rzz(rho, 0, i)
        
    # C. Measure Mobility (X-Basis)
    # If the system remains fluid, X-measurements should be random (Superposition).
    # If the system Jams (Solidifies), X-measurements should lock to a value.
    qc.h(range(1, 5))
    qc.measure(range(1, 5), range(4))
    
    circuits.append(qc)

# RUN
print("Simulating Gravitational Collapse (Jamming)...")
isa_circuits = transpile(circuits, backend=backend)
sampler = SamplerV2(mode=backend)
job = sampler.run(isa_circuits, shots=4096)
print(f"Job ID: {job.job_id()}")

result = job.result()

# ANALYZE: MOBILITY PARAMETER
print("\n=== JAMMING TRANSITION REPORT ===")
print(f"{'DENSITY':<10} | {'MOBILITY':<10} | {'PHASE'}")
print("-" * 40)

for i, rho in enumerate(densities):
    counts = result[i].data.mobility.get_counts()
    
    # Calculate "Mobility" (Entropy of the outcome)
    # High Entropy = Liquid (Fluid Vacuum)
    # Low Entropy = Solid (Frozen/Jammed Vacuum)
    
    # Simple metric: How many unique bitstrings appeared?
    # Or: Fraction of shots in the dominant state.
    
    sorted_counts = sorted(counts.values(), reverse=True)
    dominant_fraction = sorted_counts[0] / sum(counts.values())
    mobility = 1.0 - dominant_fraction
    
    phase = "LIQUID"
    if mobility < 0.1: phase = "SOLID (JAMMED)"
    elif mobility < 0.4: phase = "VISCOUS"
        
    print(f"{rho:.2f} rad   | {mobility:.3f}      | {phase}")

print("-" * 40)
print("VERDICT: If a 'SOLID' phase appears, the Singularity is prevented.")

Simulating Gravitational Collapse (Jamming)...
Job ID: d686bh1v6o8c73d6atmg

=== JAMMING TRANSITION REPORT ===
DENSITY    | MOBILITY   | PHASE
----------------------------------------
0.00 rad   | 0.002      | SOLID (JAMMED)
0.70 rad   | 0.435      | LIQUID
1.40 rad   | 0.877      | LIQUID
2.09 rad   | 0.708      | LIQUID
2.79 rad   | 0.211      | VISCOUS
3.49 rad   | 0.222      | VISCOUS
4.19 rad   | 0.713      | LIQUID
4.89 rad   | 0.884      | LIQUID
5.59 rad   | 0.437      | LIQUID
6.28 rad   | 0.000      | SOLID (JAMMED)
----------------------------------------
VERDICT: If a 'SOLID' phase appears, the Singularity is prevented.
